# Challenge 4 study - Quantized Model (4-bits everywhere)

## Import libraries and data, init virtual device

In [1]:
import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, ReLU

from akida import Device, AKD1000
import cnn2snn

device = AKD1000()

2022-11-10 22:55:35.220862: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-10 22:55:36.442255: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-10 22:55:36.442318: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-10 22:55:36.644089: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-10 22:55:45.069121: W tensorflow/stream_executor/platform/de

## Data Preparation

In [2]:
coords_ch4 = np.load('coords_ch4.npy')
color = np.load('result_ch4.npy')

# Pack in a dataframe and scale for 4-bits quantization
data = pd.DataFrame(coords_ch4, columns = ['x', 'y'])
data = round(15*data).astype(np.uint8)

# Shuffle and split train/test
data['color'] = color
data_train = data.sample(frac=0.7, axis=0)
data_test = data.drop(data_train.index)

# Split x y
x_train = data_train.drop('color', axis=1)
y_train = data_train['color']
x_test = data_test.drop('color', axis=1)
y_test = data_test['color']
data

,x,y,color
0,13,8,1
1,10,4,0
2,7,9,0
3,12,15,0
4,13,3,1
...,...,...,...
9995,14,11,0
9996,8,1,1
9997,5,7,1
9998,6,14,1


## Model creation with Keras

In [3]:
base = Sequential(name="Base_ch4")
base.add(Dense(1000, input_shape=(2,), name='FC1'))
base.add(ReLU(name='relu1'))
base.add(Dense(500, input_shape=(2,), name='FC2'))
base.add(ReLU(name='relu2'))
base.add(Dense(250, input_shape=(2,), name='FC3'))
base.add(ReLU(name='relu3'))
base.add(Dense(100, input_shape=(2,), name='FC4'))
base.add(ReLU(name='relu4'))
base.add(Dense(50, input_shape=(2,), name='FC5'))
base.add(ReLU(name='relu5'))
base.add(Dense(1, input_shape=(2,), name='FC6'))
base.add(Activation('sigmoid', name='sigmoid'))

#base.summary()

## Model quantization (4-bits everywhere)

In [4]:
qmodel = cnn2snn.quantize(base, weight_quantization=4, activ_quantization=4)
qmodel.summary()

print('\nAkida Compatibility:', cnn2snn.check_model_compatibility(qmodel, input_is_image=False))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 FC1 (QuantizedDense)        (None, 1000)              3000      
                                                                 
 relu1 (QuantizedReLU)       (None, 1000)              0         
                                                                 
 FC2 (QuantizedDense)        (None, 500)               500500    
                                                                 
 relu2 (QuantizedReLU)       (None, 500)               0         
                                                                 
 FC3 (QuantizedDense)        (None, 250)               125250    
                                                                 
 relu3 (QuantizedReLU)       (None, 250)               0         
                                                                 
 FC4 (QuantizedDense)        (None, 100)              

## Preview akida model

In [5]:
akmodel = cnn2snn.convert(qmodel, input_is_image=False)
akmodel.map(device)
akmodel.summary()

                   Model Summary                   
___________________________________________________
Input shape  Output shape  Sequences  Layers  NPs
[1, 1, 2]    [1, 1, 1]     1          7       6  
___________________________________________________

____________________________________________________
Layer (type)  Output shape  Kernel shape       NPs

==== HW/FC1-FC6 (Hardware) - size: 560364 bytes ====

FC1 (Fully.)  [1, 1, 1000]  (1, 1, 2, 1000)    1  
____________________________________________________
FC2 (Fully.)  [1, 1, 500]   (1, 1, 1000, 500)  1  
____________________________________________________
FC3 (Fully.)  [1, 1, 250]   (1, 1, 500, 250)   1  
____________________________________________________
FC4 (Fully.)  [1, 1, 100]   (1, 1, 250, 100)   1  
____________________________________________________
FC5 (Fully.)  [1, 1, 50]    (1, 1, 100, 50)    1  
____________________________________________________
FC6 (Fully.)  [1, 1, 1]     (1, 1, 50, 1)      1  
_____________

## Train quantized model

In [6]:
qmodel.compile(loss='binary_crossentropy',
            optimizer='adam',
            metrics=['accuracy']
            )
qmodel.fit(x_train, y_train, batch_size=64, epochs=600)
qmodel.evaluate(x_test, y_test)

Epoch 1/600
110/110 [==============================] - 36s 94ms/step - loss: 0.6960 - accuracy: 0.5314
Epoch 2/600
110/110 [==============================] - 10s 90ms/step - loss: 0.6836 - accuracy: 0.5479
Epoch 3/600
110/110 [==============================] - 9s 78ms/step - loss: 0.6594 - accuracy: 0.5866
Epoch 4/600
110/110 [==============================] - 9s 82ms/step - loss: 0.6602 - accuracy: 0.5764
Epoch 5/600
110/110 [==============================] - 9s 80ms/step - loss: 0.6137 - accuracy: 0.6427
Epoch 6/600
110/110 [==============================] - 9s 81ms/step - loss: 0.5848 - accuracy: 0.6804
Epoch 7/600
110/110 [==============================] - 10s 87ms/step - loss: 0.5360 - accuracy: 0.7210
Epoch 8/600
110/110 [==============================] - 9s 82ms/step - loss: 0.5415 - accuracy: 0.7163
Epoch 9/600
110/110 [==============================] - 11s 101ms/step - loss: 0.5525 - accuracy: 0.7106
Epoch 10/600
110/110 [==============================] - 11s 97ms/step - loss:

110/110 [==============================] - 11s 101ms/step - loss: 0.3650 - accuracy: 0.8407
Epoch 80/600
110/110 [==============================] - 10s 95ms/step - loss: 0.3723 - accuracy: 0.8323
Epoch 81/600
110/110 [==============================] - 11s 100ms/step - loss: 0.3585 - accuracy: 0.8344
Epoch 82/600
110/110 [==============================] - 11s 100ms/step - loss: 0.3603 - accuracy: 0.8340
Epoch 83/600
110/110 [==============================] - 11s 102ms/step - loss: 0.3660 - accuracy: 0.8340
Epoch 84/600
110/110 [==============================] - 11s 101ms/step - loss: 0.3735 - accuracy: 0.8316
Epoch 85/600
110/110 [==============================] - 11s 101ms/step - loss: 0.3677 - accuracy: 0.8297
Epoch 86/600
110/110 [==============================] - 11s 100ms/step - loss: 0.3669 - accuracy: 0.8271
Epoch 87/600
110/110 [==============================] - 11s 103ms/step - loss: 0.3442 - accuracy: 0.8451
Epoch 88/600
110/110 [==============================] - 11s 102ms/ste

110/110 [==============================] - 12s 106ms/step - loss: 0.3560 - accuracy: 0.8333
Epoch 157/600
110/110 [==============================] - 11s 100ms/step - loss: 0.3604 - accuracy: 0.8331
Epoch 158/600
110/110 [==============================] - 11s 99ms/step - loss: 0.3631 - accuracy: 0.8333
Epoch 159/600
110/110 [==============================] - 11s 102ms/step - loss: 0.3690 - accuracy: 0.8316
Epoch 160/600
110/110 [==============================] - 12s 105ms/step - loss: 0.3621 - accuracy: 0.8377
Epoch 161/600
110/110 [==============================] - 11s 101ms/step - loss: 0.3578 - accuracy: 0.8344
Epoch 162/600
110/110 [==============================] - 12s 105ms/step - loss: 0.3547 - accuracy: 0.8344
Epoch 163/600
110/110 [==============================] - 11s 103ms/step - loss: 0.3763 - accuracy: 0.8254
Epoch 164/600
110/110 [==============================] - 11s 104ms/step - loss: 0.3780 - accuracy: 0.8250
Epoch 165/600
110/110 [==============================] - 12s 

110/110 [==============================] - 11s 98ms/step - loss: 0.3665 - accuracy: 0.8380
Epoch 234/600
110/110 [==============================] - 11s 97ms/step - loss: 0.3572 - accuracy: 0.8357
Epoch 235/600
110/110 [==============================] - 11s 96ms/step - loss: 0.3703 - accuracy: 0.8330
Epoch 236/600
110/110 [==============================] - 11s 98ms/step - loss: 0.3878 - accuracy: 0.8239
Epoch 237/600
110/110 [==============================] - 11s 99ms/step - loss: 0.3643 - accuracy: 0.8339
Epoch 238/600
110/110 [==============================] - 11s 96ms/step - loss: 0.3783 - accuracy: 0.8281
Epoch 239/600
110/110 [==============================] - 11s 98ms/step - loss: 0.3702 - accuracy: 0.8304
Epoch 240/600
110/110 [==============================] - 10s 94ms/step - loss: 0.3771 - accuracy: 0.8291
Epoch 241/600
110/110 [==============================] - 11s 96ms/step - loss: 0.3695 - accuracy: 0.8324
Epoch 242/600
110/110 [==============================] - 11s 99ms/ste

110/110 [==============================] - 10s 88ms/step - loss: 0.3427 - accuracy: 0.8411
Epoch 311/600
110/110 [==============================] - 10s 89ms/step - loss: 0.3397 - accuracy: 0.8416
Epoch 312/600
110/110 [==============================] - 10s 93ms/step - loss: 0.3411 - accuracy: 0.8429
Epoch 313/600
110/110 [==============================] - 10s 94ms/step - loss: 0.3320 - accuracy: 0.8460
Epoch 314/600
110/110 [==============================] - 11s 96ms/step - loss: 0.3442 - accuracy: 0.8399
Epoch 315/600
110/110 [==============================] - 11s 97ms/step - loss: 0.3386 - accuracy: 0.8411
Epoch 316/600
110/110 [==============================] - 10s 95ms/step - loss: 0.3443 - accuracy: 0.8417
Epoch 317/600
110/110 [==============================] - 11s 97ms/step - loss: 0.3483 - accuracy: 0.8377
Epoch 318/600
110/110 [==============================] - 11s 97ms/step - loss: 0.3630 - accuracy: 0.8319
Epoch 319/600
110/110 [==============================] - 11s 96ms/ste

110/110 [==============================] - 10s 94ms/step - loss: 0.3541 - accuracy: 0.8306
Epoch 388/600
110/110 [==============================] - 11s 104ms/step - loss: 0.3563 - accuracy: 0.8327
Epoch 389/600
110/110 [==============================] - 11s 97ms/step - loss: 0.3608 - accuracy: 0.8306
Epoch 390/600
110/110 [==============================] - 11s 100ms/step - loss: 0.3728 - accuracy: 0.8253
Epoch 391/600
110/110 [==============================] - 10s 95ms/step - loss: 0.3316 - accuracy: 0.8436
Epoch 392/600
110/110 [==============================] - 11s 99ms/step - loss: 0.3426 - accuracy: 0.8366
Epoch 393/600
110/110 [==============================] - 11s 102ms/step - loss: 0.3322 - accuracy: 0.8476
Epoch 394/600
110/110 [==============================] - 11s 99ms/step - loss: 0.3526 - accuracy: 0.8337
Epoch 395/600
110/110 [==============================] - 11s 97ms/step - loss: 0.3579 - accuracy: 0.8291
Epoch 396/600
110/110 [==============================] - 10s 94ms/

110/110 [==============================] - 10s 91ms/step - loss: 0.3517 - accuracy: 0.8330
Epoch 465/600
110/110 [==============================] - 10s 90ms/step - loss: 0.3599 - accuracy: 0.8309
Epoch 466/600
110/110 [==============================] - 10s 91ms/step - loss: 0.3614 - accuracy: 0.8254
Epoch 467/600
110/110 [==============================] - 10s 94ms/step - loss: 0.3549 - accuracy: 0.8297
Epoch 468/600
110/110 [==============================] - 10s 94ms/step - loss: 0.3809 - accuracy: 0.8146
Epoch 469/600
110/110 [==============================] - 10s 93ms/step - loss: 0.3570 - accuracy: 0.8324
Epoch 470/600
110/110 [==============================] - 11s 99ms/step - loss: 0.3623 - accuracy: 0.8283
Epoch 471/600
110/110 [==============================] - 11s 97ms/step - loss: 0.3827 - accuracy: 0.8206
Epoch 472/600
110/110 [==============================] - 11s 96ms/step - loss: 0.3635 - accuracy: 0.8316
Epoch 473/600
110/110 [==============================] - 10s 94ms/ste

Epoch 542/600
110/110 [==============================] - 11s 100ms/step - loss: 0.3314 - accuracy: 0.8476
Epoch 543/600
110/110 [==============================] - 11s 98ms/step - loss: 0.3382 - accuracy: 0.8407
Epoch 544/600
110/110 [==============================] - 11s 98ms/step - loss: 0.3288 - accuracy: 0.8466
Epoch 545/600
110/110 [==============================] - 11s 98ms/step - loss: 0.3304 - accuracy: 0.8493
Epoch 546/600
110/110 [==============================] - 11s 101ms/step - loss: 0.3244 - accuracy: 0.8477
Epoch 547/600
110/110 [==============================] - 11s 97ms/step - loss: 0.3290 - accuracy: 0.8469
Epoch 548/600
110/110 [==============================] - 11s 98ms/step - loss: 0.3305 - accuracy: 0.8450
Epoch 549/600
110/110 [==============================] - 11s 102ms/step - loss: 0.3196 - accuracy: 0.8510
Epoch 550/600
110/110 [==============================] - 11s 96ms/step - loss: 0.3178 - accuracy: 0.8559
Epoch 551/600
110/110 [=============================

[0.33189862966537476, 0.8426666855812073]